In [1]:
import os
import numpy as np
import pandas as pd
#os.getcwd()
import folium
from folium import plugins
import geopandas
import geojson
import matplotlib.cm as cm
import webbrowser
import glob

In [2]:
data_input_gpx = 'data_input_gpx' 
data_processed_gpx = 'data_processed_gpx' 
data_geojson = 'data_geojson'

dir_work = '/home/craigmatthewsmith/gps_tracks'
os.chdir(dir_work)
#os.getcwd()

#os.path.isfile(gpx_file_temp)
#use_SI = True

In [3]:
# functions
def rgb2hex(c):
    hexc = '#%02x%02x%02x'%(int(c[0]*255), int(c[1]*255), int(c[2]*255))
    return(hexc)


In [4]:
geojson_file_list = glob.glob(os.path.join(data_geojson, '*.geojson'))
n_files = len(geojson_file_list)
print(n_files)                              
#print('found %s files' %(n_files))                              
#geojson_file = os.path.join(data_geojson, '2020-03-22_15-06.geojson')
#os.path.isfile(geojson_file)



263


In [5]:

# create new GeoJson objects to reduce GeoJSON data sent to Folium map as layer
f_track = lambda x: {'color': '#FC4C02', 'weight': 5} # show some color...


In [6]:
# set up Folium map
#fmap = folium.Map(tiles = None, prefer_canvas=True, disable_3d=True)

#fmap = folium.Map(tiles='Stamen Terrain', prefer_canvas=True, disable_3d=True)
fmap = folium.Map(tiles='Stamen Terrain', location=[37.862606, -121.978372], zoom_start=10) 
folium.TileLayer(tiles = 'OpenStreetMap', name='OpenStreetMap', show=False).add_to(fmap)
folium.TileLayer(tiles = 'Stamen Terrain', name='Terrain Map', show=True).add_to(fmap)
cmap = cm.get_cmap('jet') # matplotlib colormap






In [7]:
fmap

In [7]:
features_tracks = []
features_elevation = []
features_speed = []
f = 0
#for f in range(0, 3, 1):

geojson_file = geojson_file_list[f]
print('  processing f %s of %s ' %(f, n_files))

# read geojson file
with open(geojson_file, 'r') as file:
    geojson_data = geojson.load(file)


  processing f 0 of 263 


In [9]:
geojson_data['features']

[{"geometry": {"coordinates": [[-122.032592, 37.836804], [-122.032768, 37.836771]], "type": "LineString"}, "properties": {"elevation": 116.1, "slope": 0.6, "speed": 1.5}, "type": "Feature"},
 {"geometry": {"coordinates": [[-122.032768, 37.836771], [-122.032996, 37.836747]], "type": "LineString"}, "properties": {"elevation": 116.6, "slope": 2.5, "speed": 1.5}, "type": "Feature"},
 {"geometry": {"coordinates": [[-122.032996, 37.836747], [-122.033235, 37.836751]], "type": "LineString"}, "properties": {"elevation": 117.2, "slope": 2.9, "speed": 1.5}, "type": "Feature"},
 {"geometry": {"coordinates": [[-122.033235, 37.836751], [-122.033435, 37.836838]], "type": "LineString"}, "properties": {"elevation": 118.1, "slope": 3.4, "speed": 1.5}, "type": "Feature"},
 {"geometry": {"coordinates": [[-122.033435, 37.836838], [-122.033451, 37.836861]], "type": "LineString"}, "properties": {"elevation": 118.2, "slope": 4.5, "speed": 1.5}, "type": "Feature"},
 {"geometry": {"coordinates": [[-122.033451, 

In [ ]:


#geojson_data

for feature in geojson_data['features']:
    line = geojson.LineString(feature['geometry']['coordinates'])
    features_tracks.append(geojson.Feature(geometry=line))

cmin_elevation = min(feature['properties']['elevation'] for feature in geojson_data['features'])
cmax_elevation = max(feature['properties']['elevation'] for feature in geojson_data['features'])
f_elevation = lambda x: {'color': rgb2hex(cmap((x['properties']['elevation']-cmin_elevation)/(cmax_elevation-cmin_elevation))), 'weight': 5} # cmap needs normalized data
t_elevation = folium.features.GeoJsonTooltip(fields=['elevation'], aliases=['Elevation (m)'])

for feature in geojson_data['features']:
    line = geojson.LineString(feature['geometry']['coordinates'])
    elevation = feature['properties']['elevation']
    features_elevation.append(geojson.Feature(geometry=line, properties={'elevation': elevation}))

cmin_speed = min(feature['properties']['speed'] for feature in geojson_data['features'])
cmax_speed = max(feature['properties']['speed'] for feature in geojson_data['features'])
f_speed = lambda x: {'color': rgb2hex(cmap((x['properties']['speed']-cmin_speed)/(cmax_speed-cmin_speed))), 'weight': 5} # cmap needs normalized data
t_speed = folium.features.GeoJsonTooltip(fields=['speed'], aliases=['Speed '])

for feature in geojson_data['features']:
    line = geojson.LineString(feature['geometry']['coordinates'])
    speed = feature['properties']['speed']
    features_speed.append(geojson.Feature(geometry=line, properties={'speed': speed}))



In [7]:

features_tracks = []
features_elevation = []
features_speed = []
f = 0
#for f in range(0, 3, 1):
for f in range(0, n_files, 1):
    geojson_file = geojson_file_list[f]
    print('  processing f %s of %s ' %(f, n_files))
    
    # read geojson file
    with open(geojson_file, 'r') as file:
        geojson_data = geojson.load(file)

    #geojson_data

    for feature in geojson_data['features']:
        line = geojson.LineString(feature['geometry']['coordinates'])
        features_tracks.append(geojson.Feature(geometry=line))

    cmin_elevation = min(feature['properties']['elevation'] for feature in geojson_data['features'])
    cmax_elevation = max(feature['properties']['elevation'] for feature in geojson_data['features'])
    f_elevation = lambda x: {'color': rgb2hex(cmap((x['properties']['elevation']-cmin_elevation)/(cmax_elevation-cmin_elevation))), 'weight': 5} # cmap needs normalized data
    t_elevation = folium.features.GeoJsonTooltip(fields=['elevation'], aliases=['Elevation (m)'])

    for feature in geojson_data['features']:
        line = geojson.LineString(feature['geometry']['coordinates'])
        elevation = feature['properties']['elevation']
        features_elevation.append(geojson.Feature(geometry=line, properties={'elevation': elevation}))

    cmin_speed = min(feature['properties']['speed'] for feature in geojson_data['features'])
    cmax_speed = max(feature['properties']['speed'] for feature in geojson_data['features'])
    f_speed = lambda x: {'color': rgb2hex(cmap((x['properties']['speed']-cmin_speed)/(cmax_speed-cmin_speed))), 'weight': 5} # cmap needs normalized data
    t_speed = folium.features.GeoJsonTooltip(fields=['speed'], aliases=['Speed '])

    for feature in geojson_data['features']:
        line = geojson.LineString(feature['geometry']['coordinates'])
        speed = feature['properties']['speed']
        features_speed.append(geojson.Feature(geometry=line, properties={'speed': speed}))




  processing f 0 of 566 
  processing f 1 of 566 
  processing f 2 of 566 
  processing f 3 of 566 
  processing f 4 of 566 
  processing f 5 of 566 
  processing f 6 of 566 
  processing f 7 of 566 
  processing f 8 of 566 
  processing f 9 of 566 
  processing f 10 of 566 
  processing f 11 of 566 
  processing f 12 of 566 
  processing f 13 of 566 
  processing f 14 of 566 
  processing f 15 of 566 
  processing f 16 of 566 
  processing f 17 of 566 
  processing f 18 of 566 
  processing f 19 of 566 
  processing f 20 of 566 
  processing f 21 of 566 
  processing f 22 of 566 
  processing f 23 of 566 
  processing f 24 of 566 
  processing f 25 of 566 
  processing f 26 of 566 
  processing f 27 of 566 
  processing f 28 of 566 
  processing f 29 of 566 
  processing f 30 of 566 
  processing f 31 of 566 
  processing f 32 of 566 
  processing f 33 of 566 
  processing f 34 of 566 
  processing f 35 of 566 
  processing f 36 of 566 
  processing f 37 of 566 
  processing f 38 of 5

  processing f 310 of 566 
  processing f 311 of 566 
  processing f 312 of 566 
  processing f 313 of 566 
  processing f 314 of 566 
  processing f 315 of 566 
  processing f 316 of 566 
  processing f 317 of 566 
  processing f 318 of 566 
  processing f 319 of 566 
  processing f 320 of 566 
  processing f 321 of 566 
  processing f 322 of 566 
  processing f 323 of 566 
  processing f 324 of 566 
  processing f 325 of 566 
  processing f 326 of 566 
  processing f 327 of 566 
  processing f 328 of 566 
  processing f 329 of 566 
  processing f 330 of 566 
  processing f 331 of 566 
  processing f 332 of 566 
  processing f 333 of 566 
  processing f 334 of 566 
  processing f 335 of 566 
  processing f 336 of 566 
  processing f 337 of 566 
  processing f 338 of 566 
  processing f 339 of 566 
  processing f 340 of 566 
  processing f 341 of 566 
  processing f 342 of 566 
  processing f 343 of 566 
  processing f 344 of 566 
  processing f 345 of 566 
  processing f 346 of 566 
 

In [ ]:
print('appending features to map ')

geojson_data_elevation = geojson.FeatureCollection(features_elevation)
folium.GeoJson(geojson_data_elevation, style_function=f_elevation, tooltip=t_elevation, name='Elevation (m)', show=False, smooth_factor=3.0).add_to(fmap)
        
geojson_data_speed = geojson.FeatureCollection(features_speed)
folium.GeoJson(geojson_data_speed, style_function=f_speed, tooltip=t_speed, name='Speed ', show=False, smooth_factor=3.0).add_to(fmap)

geojson_data_track = geojson.FeatureCollection(features_tracks)
folium.GeoJson(geojson_data_track, style_function=f_track, name='Track only', show=True, smooth_factor=3.0).add_to(fmap)

fmap

appending features to map 


In [ ]:
# add layer control widget
folium.LayerControl(collapsed=False).add_to(fmap)

# save map to html file
fmap.fit_bounds(fmap.get_bounds())

html_file = os.path.join(dir_work, 'index.html')
if os.path.isfile(html_file):
    os.system('rm -f '+html_file)
fmap.save(html_file)
# open html file in default browser
webbrowser.open(html_file, new=2, autoraise=True)
